In [1]:
# from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
# local_embeddings = OllamaEmbeddings(model="/root/autodl-tmp/ollama_model")
embeddings = HuggingFaceEmbeddings(model_name='/root/autodl-tmp/m3e-base',
                                      model_kwargs={'device': 'cuda'})

vectorstore = Chroma(persist_directory="./chroma_db/test_02", embedding_function=embeddings)
# vectorstore = Chroma("/root/rag/chroma_db/test_01", embedding=local_embeddings)

No sentence-transformers model found with name /root/autodl-tmp/m3e-base. Creating a new one with mean pooling.
/tmp/ipykernel_995301/3353937728.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./chroma_db/test_02", embedding_function=embeddings)


In [3]:
question = ""
docs = vectorstore.similarity_search(question)
len(docs)

4

In [2]:
model = ChatOllama(
    model="qwen2.5:32b",
)

In [5]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [6]:

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

In [20]:
question = "申请专利代理费需要什么"

docs = vectorstore.similarity_search(question)

# print(docs)
# Run
chain.invoke({"context": docs, "question": question})

'申请专利代理费需要提供发票、公务卡刷卡记录单（如果使用公务卡支付）、专利代理合同。'

In [21]:
model.invoke(question).content


'申请专利过程中，如果选择通过专利代理人来帮助处理相关事宜，则可能涉及支付给专利代理人的服务费用。专利代理费用的具体构成可以因地区、事务所和案件的复杂程度等因素而异，但通常情况下包括以下几个方面：\n\n1. **初步咨询费**：部分专利代理机构可能会对初次咨询收取一定的费用，以便为客户提供详细的咨询服务。\n\n2. **文件准备及撰写费**：这涉及到发明或实用新型技术方案的理解分析、权利要求书和说明书的撰写等工作。这部分是专利申请中的核心工作之一，因此也是费用的主要组成部分。\n\n3. **审查意见答复费**：在提交申请后，如果国家知识产权局给出审查意见，需要代理人协助准备答复材料并递交至相关部门，则会产生相应的服务费用。\n\n4. **正式提交及后续处理费**：这包括向专利局提交文件时所需支付的官方费用（如检索费、审查费等），以及代理人在整个过程中提供的其他辅助性服务所收取的费用。\n\n5. **附加服务费**：根据客户的具体需求，可能还会涉及到额外的服务项目，比如进行专利检索分析、处理国际申请事务等，这些都可能会另外计收费用。\n\n具体到每个阶段或项目的收费标准及支付方式等细节问题，建议直接与拟合作的专利代理机构沟通确认。每家代理机构的服务内容和定价策略可能存在差异，在选择时需要综合考量其专业水平和服务质量等因素。'

In [ ]:
# Test Server

In [29]:
question = "A1级别的比赛有哪些"
# question = "数学建模是什么等级的比赛"

In [30]:
import requests
import json

def get_completion(prompt):
    headers = {'Content-Type': 'application/json'}
    data = {"prompt": prompt}
    response = requests.post(url='http://127.0.0.1:2048', headers=headers, data=json.dumps(data))
    # print(response)
    return response.json()

if __name__ == '__main__':
    print(get_completion(question))

{'response': 'A1级别的比赛有：\n- 中国国际“互联网+”大学生创新创业大赛\n- “挑战杯”全国大学生课外学术科技作品竞赛\n- “挑战杯”中国大学生创业计划竞赛(“创青春”全国大学生创业大赛)'}


In [31]:
model.invoke(question).content

'“A1”通常用来表示某个领域内最高级别或最顶级的比赛，但具体含义会根据不同的领域而有所区别。在汽车赛方面，“A1”曾指代一个国际性的方程式赛车系列赛——A1 Grand Prix（A1大奖赛），不过该赛事已于2010年停止举办。\n\n如果提到“电子竞技”，有时也会用“A级”或者“A1级别”来形容最高级别的比赛，但这种叫法并不是官方标准，更多是玩家或爱好者之间的非正式说法。在电竞领域中，不同游戏项目的顶级联赛通常有自己的名称，比如《英雄联盟》的世界赛、《DOTA2》的TI国际邀请赛等。\n\n因此，“A1级别”的确切含义需要根据具体上下文来判断。如果你能提供更具体的背景信息，我可以给出更为准确的回答。'

In [15]:
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [16]:
retriever = vectorstore.as_retriever()
system_prompt = (
    "你是智能助手“智多星” "
    "一个通过AI与流程自动化技术，构建覆盖“学生-教师-管理”三端的财务知识问答新系统 "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [21]:
result = rag_chain.invoke({"input": "你是谁"})

In [24]:
result["answer"]

'我是您的智能助手“智多星”，能够帮助您处理与财务知识相关的各种问题，同时也可以提供关于学生、教师和管理方面的咨询和支持。如果您有关于北京信息科技大学的任何疑问，比如学分认定、财务管理等方面的问题，都可以随时向我提问。有什么我可以帮到您的？'

In [22]:
result

{'input': '你是谁',
 'context': [Document(metadata={'creationdate': '2023-05-09T15:15:34+08:00', 'creator': 'Microsoft® Word 2016', 'moddate': '2025-02-27T19:52:32+08:00', 'page': 1, 'page_label': '2', 'producer': 'Microsoft® Word 2016', 'source': '/root/rag/rag_data/dataPDF/学分认定管理办法.pdf', 'total_pages': 8}, page_content='— 2 — \n北京信息科技大学 \n本科生创新与实践学分认定与管理办法 \n（2023 年修订） \n第一章 总 则 \n第一条  为提高学生的综合素质， 激发和培养大学生的创新\n精神、创业意识和实践动手能力，推进人才培养模式改革，经学\n校研究决定，在本科人才培养中设立创新与实践学分，并制订本\n管理办法。 \n第二条  本科生创新与实践学分是指我校全日制本科生在\n校学习期间，根据自己的特长和爱好，利用课外时间从事创新与\n实践活动， 取得具有一定创新意义的智力劳动成果或其他优秀成\n果，署名北京信息科技大学，经学校核准认定后的学分。 \n第二章 认定范围与原则 \n第三条  创新与实践学分的认定范围包括：科研成果、学术\n论文、学科竞赛、创新创业项目及其它课外实践创新活动等方面\n取得的成果。 \n第四条  创新与实践学分的认定标准见附件。 \n第五条  坚持“自主管理，本人负责”的原则，个人取得的\n创新与实践成果由个人申报， 集体取得的创新成果由指导教师负\n责组织申报。 \n第六条  学校定于每学年第二学期开展本年度本科毕业生'),
  Document(metadata={'author': 'BC6400', 'creationdate': '2024-10-22T13:46:58+08:00', 'creator': 'Microsoft® Word 2019', 'moddate': '2024-10-22T13:46:58+08:00', 'page': 71, 'page_label': '72', 'pro